In [1]:
import os
from os.path import join, splitext, basename, dirname, abspath
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn as sk
%matplotlib inline

In [4]:
import trading as trd
from trading import State, train_model, MODEL_DIR

In [ ]:
import pdb #pdb.pm()

In [5]:
# relaod module after modifying it
import importlib
importlib.reload(trd)
importlib.reload(trd.stock_history)
importlib.reload(trd.portfolio)
importlib.reload(trd.benchmarks)
importlib.reload(trd.rl)
importlib.reload(trd.nn)
importlib.reload(trd)

<module 'trading' from 'C:\\Users\\hamza\\Documents\\school\\cs_229\\cs229_final_project\\src\\trading\\__init__.py'>

## The Problem

In [ ]:
# starting cash
cash = 1e6
# starting portfolio allocation (%lo, %hi)
starting_weights = (0.5, 0.5)
# transaction cost to buy/sell a stock
trans_cost = 0.001
# number of days input
d = 7

In [7]:
# pentalty on volatility for reward method
#λ = 0.5
#reward = trd.create_penalized_returns_reward(λ)
#reward_name = 'pen_reward_l={:s}'.format(str(λ).replace('.', '_'))
reward = trd.sharpe_ratio_reward
reward_name = 'sharpe_reward'

## The Data

In [8]:
train_data, test_data = trd.get_stock_pairs(m)
portfolio_states = [State(p, cash=cash, target_weights=starting_weights, trans_cost=trans_cost, d=d) for p in train_data]

In [12]:
# size of training set
m_train = len(train_data)
m_test = len(test_data)
# total number of portfolios
M = m_train + m_test

## A Possible Solution

In [19]:
D = 6
γ = 0.99
ϵ = 0.15
H = 100
non_lin = 'relu'
τ = 0.001

In [9]:
(model, train_record) = train_model(portfolio_states, trd.actions, 
                                    D=D, gamma=γ, eps=ϵ, H=H,
                                    non_lin='relu', reward=reward, tau=τ,
                                    debug=True, debug_every=5000)

iter:        0	loss:  6.58368e+07     
iter:     5000	loss:  1.13338e+09     
iter:    10000	loss:  3.21967e+10     
iter:    15000	loss:  1.33903e+09     


## Save our Work

In [6]:
MODEL_DIR_NAME = '{:d}days_{:s}_{:d}h_{:s}_{:d}eps'.format(d, reward_name, H, non_lin, int(ϵ*100))
try:
    os.mkdir(MODEL_DIR_NAME)
    os.mkdir(join(MODEL_DIR, trd.NOTHING_DIR))
    os.mkdir(join(MODEL_DIR, trd.REBAL_DIR))
    os.mkdir(join(MODEL_DIR, trd.MAX_DIR))
    os.mkdir(join(MODEL_DIR, trd.MIN_DIR))

except Exception:
    pass

In [25]:
model.save(join(MODEL_DIR, 'model.h5'))

## Save the Test Portfolios

In [27]:
with open(join(MODEL_DIR, 'test.csv'), 'w') as f:
    for p in test_data:
        f.write('{:s},{:s}\n'.format(p.lo, p.hi))